Quitar duplicados sin tomar en cuenta la descripción del producto

# Parameters

In [ ]:
path_load = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\MEX\DB Request\Homologaciones'
filename_load = 'Homologa Farmacon S20.xlsx'

path_export = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Precios\Data\2022\Homologar'
filename_export = 'Hom_Farcon_S20.xlsx'

# Libraries

In [ ]:
import pandas as pd
import pyodbc
import random

# Connection

In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Trusted_Connection=yes;')

## Queries

In [ ]:
qweek_year = '''SELECT TmpFecha
                  ,TmpSemanaAnioGenomma Semana
                  ,TmpAnioSemanaGenomma Anio
            FROM Gnm_DWH.dbo.Dim_Tiempo'''

In [ ]:
qsemid = '''SELECT SemID
                   ,SemAnio
                   ,SemNumero
                   ,SemInicio
            FROM Gnm_MasterOp.dbo.CatSemanas'''

In [ ]:
qproducts = '''SELECT DISTINCT PR.ProPstCodBarras AS EAN
               ,PR.ProPstNombre AS Descripción
               ,PR.ProPstID
FROM Gnm_MasterOp.dbo.GnmPresentacionesProd AS PR
RIGHT JOIN Gnm_MasterOp.dbo.GnmTiposComProd AS PA
ON PA.TipoComProd = PR.TipoComProd
WHERE PA.TipoComNombre LIKE '%canal%'
      AND PR.ProPstCodBarras IN ({0})
'''

# Load Data

## Products

In [ ]:
df = pd.read_excel(
    path_load + '\\' + filename_load, 
    sheet_name='Datos', 
    header=9
)

Filtra solo las filas que necesitan homologación:

In [ ]:
df = df[['SoutRchFecha', 'SoutRchCliente', 'Cadena', 'SoutRchCodProd']][df['SoutRchRazon']=='[ProductoCliente(Unmatched)]']
df['SoutRchFecha'] = df['SoutRchFecha'].dt.date
df['SoutRchFecha'] = pd.to_datetime(df['SoutRchFecha'])
df.drop_duplicates(inplace=True)

## From DWH

Traemos años, semana y ID de semana:

In [ ]:
dweek_year = pd.read_sql(qweek_year, conn)
dsemid = pd.read_sql(qsemid, conn)

# Transformation

Obtenemos a qué semana le pertence las fechas de *SoutRchFecha*:

In [ ]:
layout = df.merge(dweek_year, left_on='SoutRchFecha', right_on='TmpFecha', how='left')
layout = layout[['SoutRchCliente', 'Anio', 'Semana', 'Cadena', 'SoutRchCodProd']]
layout.drop_duplicates(inplace=True)

En ocasiones pasan muchas semanas sin homologar un sku, por lo que no es necesario homologarlo tantas veces, lo que se necesita es homolgar una sola vez pero con la semana más antigua. <br>
La siguiente función tomará la columna *SoutRchCodProd* y revisará cuántos skus repetidos hay, después para cada sku tomará la semana más antigua y la reemplazará en las demás semanas para que al final quitemos valores duplicados. Así obtendremos valores sin repetir en *Semana* y *SoutRchCodProd*:

In [ ]:
def remove_duplicates(data, column_rep, column_fix, method='min'):
    '''
    data: Son los datos a comparar
    column_rep: Columna donde hay valores repetidos (para este caso, SoutRchCodProd o EAN)
    column_fix: Columna con los valores a imputar (para este caso, Semana o ProPstID)
    method: Solo toma el valor mínimo ('min') o máximo ('max') de column_fix
    '''
    skus_values = data[column_rep].value_counts()[data[column_rep].value_counts() > 1].index.tolist()
    for val in skus_values:
        if method == 'min':
            value_fill = data.loc[data[column_rep] == val, column_fix].min()
        else:
            value_fill = data.loc[data[column_rep] == val, column_fix].max()
        data.loc[data[column_rep] == val, column_fix] = value_fill
    return data.drop_duplicates()

Nos quedamos con el código de producto del cliente (SoutRchCodProd) único por la semana más antigua:

In [ ]:
layout = remove_duplicates(layout, column_rep='SoutRchCodProd', column_fix='Semana')

Por seguridad homologamos 2 semanas antes de que hayan aparecidos los productos. Para realizar 2 semanas hacia atrás restaremos dos semanas a la columna *SemID*:

In [ ]:
layout = layout.merge(
    dsemid, 
    left_on=['Anio', 'Semana'], 
    right_on=['SemAnio', 'SemNumero'], 
    how='left'
)

In [ ]:
layout = layout[['SemID', 'SoutRchCliente', 'Cadena', 'SoutRchCodProd']]
layout['SemID'] = layout['SemID'] - 2
layout = layout.merge(dsemid, on='SemID', how='left')

Con esto ya tenemos la información de las semanas, pero nos falta la de los productos:

In [ ]:
total_skus = ", ".join(map(str, layout['SoutRchCodProd'].unique()))
dproducts = pd.read_sql(qproducts.format(total_skus), conn)

Trateremos de quedarnos con los EAN y ProPstID únicos:

In [ ]:
dproducts = remove_duplicates(dproducts, column_rep='EAN', column_fix='ProPstID', method='max')

Aunque para el caso de los productos la función *remove_duplicates* no asegura que sean únicos, pues si hay un EAN y ProPstID con diferentes descripciones nos indica que es un registro duplicado, por lo consiguiente será asegurarnos de que sean únicos:

In [ ]:
# Creamos un catálogo de EAN y Descripción
names_sku = dproducts[['EAN', 'Descripción']].copy()
# Ahora sí nos quedamos con los productos únicos
dproducts = dproducts[['EAN', 'ProPstID']].drop_duplicates()
# Iteramos todos los renglones y asignamos nombres
dproducts['Descripción'] = pd.Series()
for row in range(dproducts.shape[0]):
    ean = dproducts.loc[row, 'EAN']
    names = names_sku['Descripción'][names_sku['EAN']==ean].tolist()
    dproducts.loc[row, 'Descripción'] = random.choice(names)
# Eliminamos el catálogo
del names_sku

In [ ]:
# Generamos el layout final
layout = layout.merge(dproducts, left_on='SoutRchCodProd', right_on='EAN', how='left')

## Formatting file

Cambiamos el nombre de las columnas para exportar

In [ ]:
layout.rename({
    'SemInicio':'SoutRchFechaCarga',
    'SemAnio':'SoutRchAnio',
    'SemNumero':'Sem'
    }, 
    axis=1, 
    inplace=True
)

In [ ]:
layout['SoutRchFecha'] = layout['SoutRchFechaCarga']
layout['Mes'] = 0

layout = layout[['SoutRchFechaCarga', 'SoutRchCliente', 'SoutRchFecha', 'SoutRchAnio', 'Mes', 'Sem', 'Cadena', 'SoutRchCodProd', 'Descripción', 'ProPstID']]

layout['Cadena'] = layout['Cadena'].map(int).map(str)
layout['SoutRchCodProd'] = layout['SoutRchCodProd'].map(int).map(str)

layout['Descripción'].fillna(value='ALTA', inplace=True)
layout['ProPstID'].fillna(value='ALTA', inplace=True)

# Export

In [ ]:
layout.to_excel(path_export + '\\' + filename_export, index=False, sheet_name='Homologar')